#Pyspark sql Exercise-1

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Employee_data").getOrCreate()

In [2]:
from pyspark.sql import Row
data = [
Row(EmpID=101, Name="Ravi", Department="Engineering", Project="AI Engine",
Salary=95000, HoursPerWeek=42),
Row(EmpID=102, Name="Sneha", Department="Engineering", Project="Data Platform",
Salary=87000, HoursPerWeek=45),
Row(EmpID=103, Name="Kabir", Department="Marketing", Project="Product Launch",
Salary=65000, HoursPerWeek=40),
Row(EmpID=104, Name="Anita", Department="Sales", Project="Client Outreach",
Salary=70000, HoursPerWeek=38),
Row(EmpID=105, Name="Divya", Department="Engineering", Project="AI Engine",
Salary=99000, HoursPerWeek=48),
Row(EmpID=106, Name="Amit", Department="Marketing", Project="Social Media",
Salary=62000, HoursPerWeek=35),
Row(EmpID=107, Name="Priya", Department="HR", Project="Policy Revamp",
Salary=58000, HoursPerWeek=37),
Row(EmpID=108, Name="Manav", Department="Sales", Project="Lead Gen", Salary=73000,
HoursPerWeek=41),
Row(EmpID=109, Name="Neha", Department="Engineering", Project="Security Suite",
Salary=91000, HoursPerWeek=46),
Row(EmpID=110, Name="Farah", Department="HR", Project="Onboarding", Salary=60000,
HoursPerWeek=36)
]
df = spark.createDataFrame(data)
df.show(truncate=False)

+-----+-----+-----------+---------------+------+------------+
|EmpID|Name |Department |Project        |Salary|HoursPerWeek|
+-----+-----+-----------+---------------+------+------------+
|101  |Ravi |Engineering|AI Engine      |95000 |42          |
|102  |Sneha|Engineering|Data Platform  |87000 |45          |
|103  |Kabir|Marketing  |Product Launch |65000 |40          |
|104  |Anita|Sales      |Client Outreach|70000 |38          |
|105  |Divya|Engineering|AI Engine      |99000 |48          |
|106  |Amit |Marketing  |Social Media   |62000 |35          |
|107  |Priya|HR         |Policy Revamp  |58000 |37          |
|108  |Manav|Sales      |Lead Gen       |73000 |41          |
|109  |Neha |Engineering|Security Suite |91000 |46          |
|110  |Farah|HR         |Onboarding     |60000 |36          |
+-----+-----+-----------+---------------+------+------------+



##Step 2: Create Views

In [3]:
#Create a Local Temp View
df.createOrReplaceTempView("employees_local")
#Create a Global Temp View
df.createOrReplaceGlobalTempView("employees_global")

##Part A: Exercises on Local View ( employees_local )

In [9]:

#1. List all employees working on the "AI Engine" project.
spark.sql("select * from employees_local where Project='AI Engine'").show()
#2. Show all employees from the "Marketing" department with salaries greater than 60,000.
spark.sql("select * from employees_local where Department='Marketing' and Salary>60000").show()
#3. Calculate the average salary for each department.
spark.sql("select Department,avg(Salary) from employees_local group by Department").show()


+-----+-----+-----------+---------+------+------------+
|EmpID| Name| Department|  Project|Salary|HoursPerWeek|
+-----+-----+-----------+---------+------+------------+
|  101| Ravi|Engineering|AI Engine| 95000|          42|
|  105|Divya|Engineering|AI Engine| 99000|          48|
+-----+-----+-----------+---------+------+------------+

+-----+-----+----------+--------------+------+------------+
|EmpID| Name|Department|       Project|Salary|HoursPerWeek|
+-----+-----+----------+--------------+------+------------+
|  103|Kabir| Marketing|Product Launch| 65000|          40|
|  106| Amit| Marketing|  Social Media| 62000|          35|
+-----+-----+----------+--------------+------+------------+

+-----------+-----------+
| Department|avg(Salary)|
+-----------+-----------+
|      Sales|    71500.0|
|Engineering|    93000.0|
|  Marketing|    63500.0|
|         HR|    59000.0|
+-----------+-----------+



In [8]:
#4. List the top 3 highest paid employees overall.
spark.sql("select Name,Salary from employees_local order by Salary desc limit 3").show()
#5. Find employees who work more than 40 hours per week.
spark.sql("select * from employees_local where HoursPerWeek>40").show()
#6. Group by project and display the number of employees per project.
spark.sql("select Project,count(EmpID) from employees_local group by Project").show()


+-----+------+
| Name|Salary|
+-----+------+
|Divya| 99000|
| Ravi| 95000|
| Neha| 91000|
+-----+------+

+-----+-----+-----------+--------------+------+------------+
|EmpID| Name| Department|       Project|Salary|HoursPerWeek|
+-----+-----+-----------+--------------+------+------------+
|  101| Ravi|Engineering|     AI Engine| 95000|          42|
|  102|Sneha|Engineering| Data Platform| 87000|          45|
|  105|Divya|Engineering|     AI Engine| 99000|          48|
|  108|Manav|      Sales|      Lead Gen| 73000|          41|
|  109| Neha|Engineering|Security Suite| 91000|          46|
+-----+-----+-----------+--------------+------+------------+

+---------------+------------+
|        Project|count(EmpID)|
+---------------+------------+
|  Data Platform|           1|
|      AI Engine|           2|
| Product Launch|           1|
|Client Outreach|           1|
| Security Suite|           1|
|  Policy Revamp|           1|
|       Lead Gen|           1|
|   Social Media|           1|
|  

In [29]:
#7. Drop the local view. Try querying again — what happens?
spark.sql("drop view employees_local")
spark.sql("select * from employees_local").show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `spark_catalog`.`default`.`employees_local` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.

##Part B: Exercises on Global View ( employees_global )

In [12]:
#6. Open a new Spark session and query "global_temp.employees_global" from there.
newspark=SparkSession.builder.appName("Employee_datagl").getOrCreate()

In [17]:
#1. Retrieve all "HR" employees working fewer than 38 hours/week.
newspark.sql("select * from global_temp.employees_global where Department='HR' and HoursPerWeek<38").show()

#2. Calculate the total salary payout for each department.
newspark.sql("select Department,sum(Salary) from global_temp.employees_global group by Department").show()


+-----+-----+----------+-------------+------+------------+
|EmpID| Name|Department|      Project|Salary|HoursPerWeek|
+-----+-----+----------+-------------+------+------------+
|  107|Priya|        HR|Policy Revamp| 58000|          37|
|  110|Farah|        HR|   Onboarding| 60000|          36|
+-----+-----+----------+-------------+------+------------+

+-----------+-----------+
| Department|sum(Salary)|
+-----------+-----------+
|      Sales|     143000|
|Engineering|     372000|
|  Marketing|     127000|
|         HR|     118000|
+-----------+-----------+



" If HoursPerWeek > 45 → 'Overworked'\nOtherwise → 'Normal"

In [19]:
#3. For each employee, add a derived column Status :
''' If HoursPerWeek > 45 → 'Overworked'
Otherwise → 'Normal'''
newspark.sql("select *, case when HoursPerWeek>45 then 'Overworked' else 'Normal' end as Status from global_temp.employees_global").show()


#4. Count the total number of employees working on each "Project" .
newspark.sql("select Project,count(EmpID) from global_temp.employees_global group by Project").show()

#5. List employees whose salary is above the average salary in their department.
newspark.sql("select * from global_temp.employees_global where Salary>(select avg(Salary) from global_temp.employees_global)").show()



+-----+-----+-----------+---------------+------+------------+----------+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|    Status|
+-----+-----+-----------+---------------+------+------------+----------+
|  101| Ravi|Engineering|      AI Engine| 95000|          42|    Normal|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|    Normal|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|    Normal|
|  104|Anita|      Sales|Client Outreach| 70000|          38|    Normal|
|  105|Divya|Engineering|      AI Engine| 99000|          48|Overworked|
|  106| Amit|  Marketing|   Social Media| 62000|          35|    Normal|
|  107|Priya|         HR|  Policy Revamp| 58000|          37|    Normal|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|    Normal|
|  109| Neha|Engineering| Security Suite| 91000|          46|Overworked|
|  110|Farah|         HR|     Onboarding| 60000|          36|    Normal|
+-----+-----+-----------+---------------+------+---

#Bonus Challenges

In [26]:
#1. Use a window function to assign rank to employees within each department based on salary.
from pyspark.sql import Window
from pyspark.sql.functions import rank
window_spec = Window.partitionBy("Department").orderBy(df.Salary.desc())
newspark.sql("select *, rank() over (partition by Department order by Salary desc) as SalaryRank from global_temp.employees_global").show()



+-----+-----+-----------+---------------+------+------------+----------+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|SalaryRank|
+-----+-----+-----------+---------------+------+------------+----------+
|  105|Divya|Engineering|      AI Engine| 99000|          48|         1|
|  101| Ravi|Engineering|      AI Engine| 95000|          42|         2|
|  109| Neha|Engineering| Security Suite| 91000|          46|         3|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|         4|
|  110|Farah|         HR|     Onboarding| 60000|          36|         1|
|  107|Priya|         HR|  Policy Revamp| 58000|          37|         2|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|         1|
|  106| Amit|  Marketing|   Social Media| 62000|          35|         2|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|         1|
|  104|Anita|      Sales|Client Outreach| 70000|          38|         2|
+-----+-----+-----------+---------------+------+---

In [27]:
#2. Create another view (local or global) that only contains "Engineering" employees.
newspark.sql("create or replace Temp view engineering_employees as select * from global_temp.employees_global where Department='Engineering'")
newspark.sql("select*from engineering_employees").show()

+-----+-----+-----------+--------------+------+------------+
|EmpID| Name| Department|       Project|Salary|HoursPerWeek|
+-----+-----+-----------+--------------+------+------------+
|  101| Ravi|Engineering|     AI Engine| 95000|          42|
|  102|Sneha|Engineering| Data Platform| 87000|          45|
|  105|Divya|Engineering|     AI Engine| 99000|          48|
|  109| Neha|Engineering|Security Suite| 91000|          46|
+-----+-----+-----------+--------------+------+------------+



In [28]:
#3. Create a SQL view that filters out all employees working < 38 hours and savesit as "active_employees" .
newspark.sql("create or replace Temp view active_employees as select * from global_temp.employees_global where HoursPerWeek>38")
newspark.sql("select*from active_employees").show()


+-----+-----+-----------+--------------+------+------------+
|EmpID| Name| Department|       Project|Salary|HoursPerWeek|
+-----+-----+-----------+--------------+------+------------+
|  101| Ravi|Engineering|     AI Engine| 95000|          42|
|  102|Sneha|Engineering| Data Platform| 87000|          45|
|  103|Kabir|  Marketing|Product Launch| 65000|          40|
|  105|Divya|Engineering|     AI Engine| 99000|          48|
|  108|Manav|      Sales|      Lead Gen| 73000|          41|
|  109| Neha|Engineering|Security Suite| 91000|          46|
+-----+-----+-----------+--------------+------+------------+

